In [ ]:
import pandas as pd
import numpy as np
from pingouin import cronbach_alpha
from semopy import Model

# Load your dataset
data = pd.read_csv('path_to_your_data.csv')

# Define your model based on the constructs and items
model_desc = """
# Example of model specification, adjust based on actual items
JobCrafting =~ jc1 + jc2 + jc3 + jc4
Openness =~ open1 + open2 + open3 + open4
Conscientiousness =~ consc1 + consc2 + consc3 + consc4
Extraversion =~ extra1 + extra2 + extra3 + extra4
Agreeableness =~ agree1 + agree2 + agree3 + agree4
Neuroticism =~ neuro1 + neuro2 + neuro3 + neuro4
GenAIUse =~ gai1 + gai2 + gai3 + gai4
"""

# Initialize and fit the model
mod = Model(model_desc)
mod.fit(data)

# Get estimated parameters
params = mod.inspect()

# Calculate Composite Reliability (CR)
def calculate_composite_reliability(params, construct):
    # Extract factor loadings for the construct
    loadings = params.loc[(params['op'] == '~') & (params['lval'].str.contains(f'^{construct}$')), 'Estimate'].values
    # Calculate squared loadings (numerator)
    lambda_squared = np.sum(loadings ** 2)
    # Calculate error variances for each item (denominator part)
    error_variances = 1 - (loadings ** 2)
    vartheta_sum = np.sum(error_variances)
    # Calculate CR
    cr = lambda_squared / (lambda_squared + vartheta_sum)
    return cr

# Loop through constructs to calculate Cronbach's Alpha and CR
constructs = ['JobCrafting', 'Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism', 'GenAIUse']
for construct in constructs:
    items = [col for col in data.columns if construct.lower() in col.lower()]
    alpha = cronbach_alpha(data[items])[0]
    print(f"Cronbach's Alpha for {construct}: {alpha:.3f}")
    
    cr = calculate_composite_reliability(params, construct)
    print(f"Composite Reliability for {construct}: {cr:.3f}")
